In [1]:

import random

import evaluate
import numpy as np
import torch
from datasets import load_from_disk
from transformers import (AutoTokenizer, TrainingArguments, DataCollatorWithPadding,
                          Trainer, AutoModelForSequenceClassification, EarlyStoppingCallback)
from sklearn.model_selection import ParameterGrid
from rebert.model import ReBertModel
import gc

seed = 42
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

2024-01-05 16:51:10.017531: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-05 16:51:10.017561: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-05 16:51:10.018617: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-05 16:51:10.023663: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-01-05 16:51:10.631377: W tensorflow/compiler/tf2

In [2]:
device = "cuda"
id2label = {0: "Entailment", 1: "Neutral", 2: "Contradiction"}
label2id = {v: k for k, v in id2label.items()}
ds = load_from_disk("./data/mnli_roberta")
ds

DatasetDict({
    train: Dataset({
        features: ['label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 392702
    })
    test_matched: Dataset({
        features: ['label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 9796
    })
    test_mismatched: Dataset({
        features: ['label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 9847
    })
    eval: Dataset({
        features: ['label', 'idx', 'input_ids', 'attention_mask'],
        num_rows: 19647
    })
})

In [3]:
accuracy = evaluate.load("accuracy")


def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = accuracy.compute(predictions=predictions, references=labels)["accuracy"]
    return {"accuracy": acc}

In [4]:
BATCH_EVAL = 128
LAMBDA = 0.01
OUTPUT = "rebert_rope_mnli"
TB_DIR = "rebert_rope_mnli_tb"

hyper_params = {
    "batch": [16, 32, 64],
    "lr": [1e-3, 5e-4, 1e-4, 2e-5]
}

tokenizer = AutoTokenizer.from_pretrained("./rebert-base")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
eval_result = []

for params in ParameterGrid(hyper_params):
    model = AutoModelForSequenceClassification.from_pretrained(
        "./rebert-base", num_labels=len(id2label), id2label=id2label, label2id=label2id
    ).to(device)

    max_steps = 2000 * 32 // params["batch"]
    training_args = TrainingArguments(
        output_dir=OUTPUT,
        learning_rate=params["lr"],
        per_device_train_batch_size=params["batch"],
        per_device_eval_batch_size=BATCH_EVAL,
        bf16=True,
        gradient_checkpointing=True,
        max_steps=max_steps,
        weight_decay=LAMBDA,
        lr_scheduler_type="constant",
        evaluation_strategy="no",
        save_strategy="no",
        seed=seed
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=ds["train"],
        eval_dataset=ds["eval"],
        tokenizer=tokenizer,
        data_collator=data_collator,
        compute_metrics=compute_metrics
    )

    results = trainer.train()
    eval_perf = trainer.evaluate()
    eval_result.append((params, eval_perf))
    
    gc.collect()
    torch.cuda.empty_cache()

Some weights of ReBertForSequenceClassification were not initialized from the model checkpoint at ./rebert-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/sdai/.conda/envs/mdbert/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences betwe

Step,Training Loss
500,1.153000
1000,1.123800
1500,1.118000
2000,1.115700
2500,1.110000
3000,1.105100
3500,1.105000
4000,1.103600


Some weights of ReBertForSequenceClassification were not initialized from the model checkpoint at ./rebert-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters: {'batch': 16, 'freeze_embeddings': True, 'lr': 0.001}
Eval: {'eval_loss': 1.0996252298355103, 'eval_accuracy': 0.3182165216063521, 'eval_runtime': 16.4935, 'eval_samples_per_second': 1191.195, 'eval_steps_per_second': 9.337, 'epoch': 0.16}


/home/sdai/.conda/envs/mdbert/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


...

Some weights of ReBertForSequenceClassification were not initialized from the model checkpoint at ./rebert-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Parameters: {'batch': 64, 'freeze_embeddings': False, 'lr': 0.0001}
Eval: {'eval_loss': 1.0554519891738892, 'eval_accuracy': 0.4482618211431771, 'eval_runtime': 16.6328, 'eval_samples_per_second': 1181.218, 'eval_steps_per_second': 9.259, 'epoch': 0.16}


/home/sdai/.conda/envs/mdbert/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss
500,1.018300
1000,0.933000


Parameters: {'batch': 64, 'freeze_embeddings': False, 'lr': 2e-05}
Eval: {'eval_loss': 0.886547863483429, 'eval_accuracy': 0.5980556828014455, 'eval_runtime': 16.6838, 'eval_samples_per_second': 1177.606, 'eval_steps_per_second': 9.23, 'epoch': 0.16}


In [5]:
best_params = sorted(eval_result, key=lambda x: x[1]["eval_loss"])
best_params[:5]

[({'batch': 32, 'freeze_embeddings': True, 'lr': 2e-05},
  {'eval_loss': 0.8783003687858582,
   'eval_accuracy': 0.6039089937395022,
   'eval_runtime': 15.9369,
   'eval_samples_per_second': 1232.799,
   'eval_steps_per_second': 9.663,
   'epoch': 0.16}),
 ({'batch': 32, 'freeze_embeddings': False, 'lr': 2e-05},
  {'eval_loss': 0.8783003687858582,
   'eval_accuracy': 0.6039089937395022,
   'eval_runtime': 16.6518,
   'eval_samples_per_second': 1179.875,
   'eval_steps_per_second': 9.248,
   'epoch': 0.16}),
 ({'batch': 64, 'freeze_embeddings': True, 'lr': 2e-05},
  {'eval_loss': 0.886547863483429,
   'eval_accuracy': 0.5980556828014455,
   'eval_runtime': 16.6818,
   'eval_samples_per_second': 1177.749,
   'eval_steps_per_second': 9.232,
   'epoch': 0.16}),
 ({'batch': 64, 'freeze_embeddings': False, 'lr': 2e-05},
  {'eval_loss': 0.886547863483429,
   'eval_accuracy': 0.5980556828014455,
   'eval_runtime': 16.6838,
   'eval_samples_per_second': 1177.606,
   'eval_steps_per_second': 9.2

In [ ]:
BATCH_TRAIN = best_params[0][0]["batch"]
BATCH_EVAL = 128
LEARNING_RATE = best_params[0][0]["lr"]
EPOCHS = 10
SAVE_STEPS = 2000
LOG_STEPS = 2000
LAMBDA = 0.01
SAVE_LIMITS = 10
WARMUP = 0.06
OUTPUT = "rebert_scratch_mnli"
TB_DIR = "rebert_scratch_mnli_tb"

tokenizer = AutoTokenizer.from_pretrained("./rebert-base")
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)
model = AutoModelForSequenceClassification.from_pretrained(
        "./rebert-base", num_labels=len(id2label), id2label=id2label, label2id=label2id
    ).to(device)

training_args = TrainingArguments(
    output_dir=OUTPUT,
    learning_rate=LEARNING_RATE,
    per_device_train_batch_size=BATCH_TRAIN,
    per_device_eval_batch_size=BATCH_EVAL,
    bf16=True,
    gradient_checkpointing=True,
    num_train_epochs=EPOCHS,
    weight_decay=LAMBDA,
    lr_scheduler_type="linear",
    warmup_ratio=WARMUP,
    evaluation_strategy="steps",
    save_strategy="steps",
    logging_steps=LOG_STEPS,
    save_steps=SAVE_STEPS,
    logging_dir=TB_DIR,
    save_total_limit=SAVE_LIMITS,
    load_best_model_at_end=True,
    seed=seed
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ds["train"],
    eval_dataset=ds["eval"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=5)]
)

results = trainer.train(resume_from_checkpoint=False)

Some weights of ReBertForSequenceClassification were not initialized from the model checkpoint at ./rebert-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/home/sdai/.conda/envs/mdbert/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences betwe

Step,Training Loss,Validation Loss,Accuracy
2000,1.058000,0.930387,0.565023
4000,0.926500,0.875809,0.606759
6000,0.883900,0.838052,0.633023
8000,0.864800,0.821480,0.634295
10000,0.842400,0.828421,0.646460
12000,0.829200,0.798516,0.653280
14000,0.802300,0.764684,0.667990
16000,0.792900,0.763899,0.671349
18000,0.782200,0.753154,0.675421
20000,0.772400,0.752355,0.677152


/home/sdai/.conda/envs/mdbert/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/sdai/.conda/envs/mdbert/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/sdai/.conda/envs/mdbert/lib/python3.10/site-pa